In [2]:
# pip install domolibrary

In [6]:
import dotenv
import os

dotenv.load_dotenv('env.txt')
domo_username = os.environ.get('DOMO_USERNAME')
domo_instance = os.environ.get('DOMO_INSTANCE')
domo_password = os.environ.get('DOMO_PASSWORD')



In [8]:
import domolibrary.client.DomoAuth as dmda

auth = dmda.DomoFullAuth(domo_username = domo_username, 
                         domo_password = domo_password, 
                         domo_instance = domo_instance)

await auth.print_is_token()

🎉 token retrieved from domo-community ⚙️


True

In [14]:
import domolibrary.classes.DomoDatacenter as dmdc

domo_datacenter = dmdc.DomoDatacenter(auth = auth)

dataflows_ls = await domo_datacenter.search_datacenter(
    auth= auth,
    entity_type = 'DATAFLOW',
    additional_filters_ls = [{"filterType":"term","field":"data_flow_type","value":"MAGIC","name":"Magic ETL v2","not":False}]
)

dataflows_ls[0]


{'entityType': 'dataflow',
 'databaseId': '227',
 'searchId': {'indexName': None,
  'databaseId': '227',
  'customer': 'mmmm-0012-0200',
  'entityType': 'dataflow'},
 'createDate': 1686863564000,
 'lastModified': 1699722570000,
 'lastIndexed': 1700700196369,
 'highlightedFields': {},
 'language': 'English',
 'requestAccess': False,
 'score': 57.957905,
 'name': 'DDX_Landing Page_QA',
 'description': '',
 'ownedById': '1893952720',
 'ownedByName': 'Jae Wilson1',
 'tags': [],
 'lastRunDate': 1686863796000,
 'inputDatasets': [{'name': 'monit_user_accesslist',
   'id': '6ddbcb8d-0f38-48ad-bd73-f6b35c4b7daf'}],
 'outputDatasets': [{'name': 'DDX_Landing Page_QA',
   'id': '51260f1d-b146-4dff-a1d8-68b7aa0b740b'}],
 'runCount': 2,
 'successRate': 1.0,
 'dataFlowType': 'MAGIC',
 'status': 'SUCCESS',
 'deleted': False,
 'passwordProtected': False,
 'abandoned': False,
 'owners': [{'id': '1893952720',
   'type': 'USER',
   'displayName': 'Jae Wilson1'}],
 'ownersLocalized': {'localizedMessage': '

In [18]:
dataflow_ids = [dataflow.get('databaseId') for dataflow in dataflows_ls]
dataflow_ids[0:5]

['227', '131', '37', '185', '38']

In [30]:
import domolibrary.classes.DomoDataflow as dmdf
import asyncio

dataflow_definition_ls = await asyncio.gather( *[dmdf.DomoDataflow.get_from_id(dataflow_id = dataflow_id, auth = auth) for dataflow_id in dataflow_ids])
dataflow_definition_ls[0]

DomoDataflow(id=227, name='DDX_Landing Page_QA', auth=DomoFullAuth(domo_instance='domo-community', domo_username='jae@onyxreporting.com', token_name='full_auth', is_valid_token=True, url_manual_login='https://domo-community.domo.com/auth/index?domoManualLogin=true'), owner=None, description=None, tags=None, actions=[DomoDataflow_Action(type='LoadFromVault', id='b9bb0f01-2517-4595-a077-c9778e5ffc48', name='monit_user_accesslist', data_source_id='6ddbcb8d-0f38-48ad-bd73-f6b35c4b7daf', sql=None), DomoDataflow_Action(type='WindowAction', id='52f47029-06e0-4599-9ed8-21c4b9386c89', name='Rank & Window', data_source_id=None, sql=None), DomoDataflow_Action(type='ExpressionEvaluator', id='623883e9-bdc1-44cd-9fb4-6277d0e123de', name='Add Formula', data_source_id=None, sql=None), DomoDataflow_Action(type='Metadata', id='99ad5978-9a19-41e1-99d7-3d0ce58f7467', name='Alter Columns', data_source_id=None, sql=None), DomoDataflow_Action(type='SelectValues', id='4cf42103-c52d-4b92-b26f-63832ead11ee', na

In [31]:
dataflow_definition_ls[0].actions

[DomoDataflow_Action(type='LoadFromVault', id='b9bb0f01-2517-4595-a077-c9778e5ffc48', name='monit_user_accesslist', data_source_id='6ddbcb8d-0f38-48ad-bd73-f6b35c4b7daf', sql=None),
 DomoDataflow_Action(type='WindowAction', id='52f47029-06e0-4599-9ed8-21c4b9386c89', name='Rank & Window', data_source_id=None, sql=None),
 DomoDataflow_Action(type='ExpressionEvaluator', id='623883e9-bdc1-44cd-9fb4-6277d0e123de', name='Add Formula', data_source_id=None, sql=None),
 DomoDataflow_Action(type='Metadata', id='99ad5978-9a19-41e1-99d7-3d0ce58f7467', name='Alter Columns', data_source_id=None, sql=None),
 DomoDataflow_Action(type='SelectValues', id='4cf42103-c52d-4b92-b26f-63832ead11ee', name='Select Columns', data_source_id=None, sql=None),
 DomoDataflow_Action(type='ExpressionEvaluator', id='435803f2-08a4-42b3-8eb2-295b1519c85f', name='Add Formula 1', data_source_id=None, sql=None),
 DomoDataflow_Action(type='PublishToVault', id='0c0f0aa1-06b7-4ec1-a3bb-2b28c9a28fa7', name='DDX_Landing Page_QA',

In [33]:

def get_action_list(domo_dataflow: dmdf.DomoDataflow):
    return [{'dataflow_id': domo_dataflow.id,
                 'datafow_name': domo_dataflow.name,
                 'action_type': domo_action.type} for domo_action in domo_dataflow.actions]

get_action_list(domo_dataflow = dataflow_definition_ls[0])

[{'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'LoadFromVault'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'WindowAction'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'ExpressionEvaluator'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'Metadata'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'SelectValues'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'ExpressionEvaluator'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'PublishToVault'}]

In [34]:

action_ls = [get_action_list(dd_obj) for dd_obj in dataflow_definition_ls]

flat_action_ls = [ action for a_list in action_ls for action in a_list]
flat_action_ls[0:5]

[{'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'LoadFromVault'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'WindowAction'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'ExpressionEvaluator'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'Metadata'},
 {'dataflow_id': 227,
  'datafow_name': 'DDX_Landing Page_QA',
  'action_type': 'SelectValues'}]

In [35]:
import pandas as pd

df = pd.DataFrame(flat_action_ls)
df[0:5]

,dataflow_id,datafow_name,action_type
0,227,DDX_Landing Page_QA,LoadFromVault
1,227,DDX_Landing Page_QA,WindowAction
2,227,DDX_Landing Page_QA,ExpressionEvaluator
3,227,DDX_Landing Page_QA,Metadata
4,227,DDX_Landing Page_QA,SelectValues


In [ ]:
df.to_csv('marks_csv.csv')